<a href="https://colab.research.google.com/github/pratyushlohumi26/time_series_forecasting/blob/main/time_series_forecasting_LSTM_XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP EXERCISE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Processing


In [ ]:
drive_path = '/content/drive/MyDrive/data/data'

data_path = '/workspace/pl/data'

In [ ]:
# !pip install tensorflow numpy pandas 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## loading the dataset from the disk

train_df = pd.read_csv(data_path + '/Train_Data.csv') 
test = pd.read_csv(data_path + '/Test_Data.csv')
submit = pd.read_csv(data_path + '/Submit.csv')
submit.head(2)

,Month,Subject,Count
0,May - 19,Astronomy and Astrophysics,0
1,May - 19,Astrophysics,0


In [ ]:
## converting String column to list of string column

train_df['Subjects'] = train_df['Subjects'].apply(lambda x: x.strip("[]").replace("'", "").split(",")) 

In [ ]:
# use explode to create a new row for each item in the Column Subject, dividing the subjects into individual rows
train_new_df = train_df.explode('Subjects')
train_new_df

,Title,Date,Abstract,Subjects
0,The Neutrino Ball Model of a Quasar,1993-04-07,It is suggested that the nonorthodox model o...,Astrophysics
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,We consider charged boson stars and study th...,Astrophysics
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,We consider charged boson stars and study th...,General Relativity and Quantum Cosmology
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,We derive lithium abundances in 53 T Tauri s...,Astrophysics
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,Recent high energy gamma-ray observations (E...,Astrophysics
...,...,...,...,...
29187,Can dark energy be expressed as a power series...,2019-04-30,In this work we examine the possibility that...,High Energy Physics - Phenomenology
29188,Slim accretion disks: theory and observational...,2019-04-30,Slim accretion disks idea emerged over 30 ye...,Astrophysics - Astrophysics of Galaxies
29188,Slim accretion disks: theory and observational...,2019-04-30,Slim accretion disks idea emerged over 30 ye...,Astrophysics - High Energy Astrophysical Phen...
29189,"A 10,000-solar-mass black hole in the nucleus ...",2019-04-30,The motions of gas and stars in the nuclei o...,Astrophysics - Astrophysics of Galaxies


In [ ]:
# Define a function to remove 'and' from the start or end of a string
def remove_and(s):
    if s.startswith("and"):
        s = s[len("and"):].strip()
    elif s.endswith("and"):
        s = s[:-len("and")].strip()
    return s

# Apply the function to the 'subject' column using the apply() method
train_new_df['Subjects'] = train_new_df['Subjects'].apply(remove_and)
train_new_df

,Title,Date,Abstract,Subjects
0,The Neutrino Ball Model of a Quasar,1993-04-07,It is suggested that the nonorthodox model o...,Astrophysics
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,We consider charged boson stars and study th...,Astrophysics
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,We consider charged boson stars and study th...,General Relativity and Quantum Cosmology
2,Pre-main sequence lithium burning. I. Weak T T...,1993-08-31,We derive lithium abundances in 53 T Tauri s...,Astrophysics
3,Very High Energy Gamma-Rays from AGN: Cascadin...,1993-12-18,Recent high energy gamma-ray observations (E...,Astrophysics
...,...,...,...,...
29187,Can dark energy be expressed as a power series...,2019-04-30,In this work we examine the possibility that...,High Energy Physics - Phenomenology
29188,Slim accretion disks: theory and observational...,2019-04-30,Slim accretion disks idea emerged over 30 ye...,Astrophysics - Astrophysics of Galaxies
29188,Slim accretion disks: theory and observational...,2019-04-30,Slim accretion disks idea emerged over 30 ye...,Astrophysics - High Energy Astrophysical Phen...
29189,"A 10,000-solar-mass black hole in the nucleus ...",2019-04-30,The motions of gas and stars in the nuclei o...,Astrophysics - Astrophysics of Galaxies


In [ ]:
import pandas as pd
import numpy as np
# from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load data

train_data = train_new_df
# test_data = test


train_data['Subjects'] = train_data['Subjects'].str.strip() ## triming any white space that might be available

print("Train data shape : ", train_data.shape)

# Preprocess data
train_data['Date'] = pd.to_datetime(train_data['Date']) ## converting String column to datetime column


## extracting year and month from date column
train_data['Year'] = train_data['Date'].apply(lambda x: x.strftime('%Y'))
train_data['Month'] = train_data['Date'].apply(lambda x: x.strftime('%m'))


## keeping only the necessary columns

train_data = train_data.drop(['Date', 'Title','Abstract'], axis=1)

train_data = train_data.groupby(['Year','Month', 'Subjects'])["Year"].count().reset_index(name="Count")

subjects = train_data['Subjects'].unique()

print("FINAL Train data shape : ", train_data.shape)

train_data

2023-04-10 07:25:59.705857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Train data shape :  (47619, 4)
FINAL Train data shape :  (2476, 4)


,Year,Month,Subjects,Count
0,1993,04,Astrophysics,1
1,1993,05,Astrophysics,1
2,1993,05,General Relativity and Quantum Cosmology,1
3,1993,08,Astrophysics,1
4,1993,12,Astrophysics,1
...,...,...,...,...
2471,2019,04,Statistics - Machine Learning,3
2472,2019,04,Statistics - Methodology,1
2473,2019,04,Statistics - Other Statistics,1
2474,2019,04,Statistics and Probability,3


## LSTM Model Training

In [ ]:
## Method 1.1 --> Using one hot encoding and lstm model

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.losses import mean_squared_error
# Encode the subject column using label encoding
le = LabelEncoder()
ohe = OneHotEncoder(categories='auto', sparse=False,handle_unknown='ignore')
def data_preprocess_one_hot_encoding(df):
    df['subject_encoded'] = le.fit_transform(df['Subjects'])

    # Encode the subject column using one-hot encoding

    
    subject_ohe = ohe.fit_transform(df['subject_encoded'].values.reshape(-1,1))

    # Combine the encoded features into a single feature matrix

    features = np.hstack((df[['Year', 'Month']].values, subject_ohe))

    # Extract the target variable

    target = df['Count'].values
    features = np.asarray(features).astype(np.float32)
    target = np.asarray(target).astype(np.float32)
    return features,target

x_one_hot,y_one_hot = data_preprocess_one_hot_encoding(train_data)

# Split the data into training and testing sets
split_index = int(0.9*len(train_data))
x_train_one_hot, x_val_one_hot = x_one_hot[:split_index], x_one_hot[split_index:]
y_train_one_hot, y_val_one_hot = y_one_hot[:split_index], y_one_hot[split_index:]


num_features = x_train_one_hot.shape[1]


In [ ]:



# Define LSTM network architecture
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(num_features, 1), return_sequences=True),
    tf.keras.layers.LSTM(32, dropout=0.1, recurrent_dropout=0.2,return_sequences=True),
    tf.keras.layers.LSTM(16, dropout=0.1, recurrent_dropout=0.2,return_sequences=True),
    tf.keras.layers.Dense(1)
])



# defining rmse loss function in keras

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.math.squared_difference(y_true, y_pred)))#tf.sqrt(tf.losses.mean_squared_error(y_true, y_pred))

# Compile the model with mean squared error loss
model.compile(optimizer='adam', loss=root_mean_squared_error)

# Define the early stopping criteria - do remove overfitting

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model using the training data
model.fit(x_train_one_hot, y_train_one_hot, epochs=100, batch_size=16,
          validation_data=(x_val_one_hot, y_val_one_hot), callbacks=[early_stopping])



2023-04-10 07:17:11.964755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-10 07:17:11.978061: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-10 07:17:12.099471: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

Epoch 1/100


2023-04-10 07:17:12.486200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-10 07:17:12.487154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-10 07:17:12.487848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

140/140 [==============================] - ETA: 0s - loss: 42.9376

2023-04-10 07:17:23.118519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-10 07:17:23.119376: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-10 07:17:23.120003: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

140/140 [==============================] - 11s 60ms/step - loss: 42.9376 - val_loss: 69.8492
Epoch 2/100
140/140 [==============================] - 8s 57ms/step - loss: 43.2545 - val_loss: 69.6973
Epoch 3/100
140/140 [==============================] - 8s 57ms/step - loss: 43.3421 - val_loss: 69.6064
Epoch 4/100
140/140 [==============================] - 8s 57ms/step - loss: 42.7831 - val_loss: 69.5602
Epoch 5/100
140/140 [==============================] - 8s 57ms/step - loss: 42.8264 - val_loss: 69.5375
Epoch 6/100
140/140 [==============================] - 8s 57ms/step - loss: 42.9975 - val_loss: 69.5276
Epoch 7/100
140/140 [==============================] - 8s 57ms/step - loss: 42.5055 - val_loss: 69.5321
Epoch 8/100
140/140 [==============================] - 8s 57ms/step - loss: 42.9714 - val_loss: 69.5261
Epoch 9/100
140/140 [==============================] - 8s 57ms/step - loss: 42.3386 - val_loss: 69.5138
Epoch 10/100
140/140 [==============================] - 8s 57ms/step - loss

In [ ]:
# Evaluate the model using the testing data
# Make predictions on the test data
from sklearn.metrics import mean_squared_error
rmse = model.evaluate(x_val_one_hot,y_val_one_hot)

print("RMSE for one hot encoded data for LSTM:", rmse)


8/8 [==============================] - 0s 13ms/step - loss: 79.8788
RMSE for one hot encoded data for LSTM: 79.87882232666016


In [ ]:
# method 1.2 using embedding layer in the lstm model 

## this helps reduce the dimensionality of the dataset and helps us to skip the one hot encoding step

from keras.layers import Dense, LSTM, Embedding, Concatenate, Input
from keras.callbacks import EarlyStopping
from tensorflow.keras import Model

def data_preprocess_wo_one_hot_encoding(df):
    df['subject_encoded'] = le.fit_transform(df['Subjects'])

    features = df[['Year', 'Month','subject_encoded']].values

    # Extract the target variable

    target = df['Count'].values
    features = np.asarray(features).astype(np.float32)
    target = np.asarray(target).astype(np.float32)
    return features,target

x,y = data_preprocess_wo_one_hot_encoding(train_data)

# Split the data into training and testing sets
split_index = int(0.9*len(train_data))
x_train, x_val = x[:split_index], x[split_index:]
y_train, y_val = y[:split_index], y[split_index:]


In [ ]:

# Define the input shapes
year_month_input = Input(shape=(2,))
subject_input = Input(shape=(1,))

# Define the embedding layer for the subject column
subject_embedding = Embedding(input_dim=120, output_dim=64)(subject_input) ## emedding layer with 32 dimension output
subject_embedding = LSTM(64, dropout=0.1, recurrent_dropout=0.2,return_sequences=True)(subject_embedding)
subject_embedding = LSTM(32, dropout=0.1, recurrent_dropout=0.1,return_sequences=True)(subject_embedding)
subject_embedding = LSTM(16, dropout=0.1, recurrent_dropout=0.1)(subject_embedding)

# Concatenate the input layers
concatenated_input = Concatenate()([year_month_input, subject_embedding])

# Define the output layer
output = Dense(1)(concatenated_input)

# Define the model
model2 = Model(inputs=[year_month_input, subject_input], outputs=output)

from keras.optimizers import Adam

# Define the optimizer with custom parameters
adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.99, epsilon=1e-8)

#  defining the rmse loss function

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

# Compile the model with mean squared error loss

model2.compile(optimizer='adam', loss=root_mean_squared_error)

# Compile the model

model2.compile(loss=root_mean_squared_error, optimizer=adam)

# Define the early stopping criteria - do remove overfitting

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model2.fit(x=[x_train[:,0:2], x_train[:,2].reshape(-1,1)], y=y_train, epochs=100, batch_size=16, 
          validation_data=([x_val[:,0:2],x_val[:,2]], y_val), callbacks=[early_stopping])



Epoch 1/100
140/140 [==============================] - 4s 5ms/step - loss: 23.1272 - val_loss: 36.9451
Epoch 2/100
140/140 [==============================] - 0s 2ms/step - loss: 16.4829 - val_loss: 36.8386
Epoch 3/100
140/140 [==============================] - 0s 1ms/step - loss: 16.3170 - val_loss: 37.8925
Epoch 4/100
140/140 [==============================] - 0s 2ms/step - loss: 15.9995 - val_loss: 36.3725
Epoch 5/100
140/140 [==============================] - 0s 1ms/step - loss: 16.2549 - val_loss: 38.8368
Epoch 6/100
140/140 [==============================] - 0s 2ms/step - loss: 16.5385 - val_loss: 38.8026
Epoch 7/100
140/140 [==============================] - 0s 2ms/step - loss: 16.0850 - val_loss: 36.6027
Epoch 8/100
140/140 [==============================] - 0s 1ms/step - loss: 16.0314 - val_loss: 36.6023
Epoch 9/100
140/140 [==============================] - 0s 1ms/step - loss: 16.7754 - val_loss: 36.2468
Epoch 10/100
140/140 [==============================] - 0s 1ms/step - los

In [ ]:
# Evaluate the model on the test set
# Make predictions on the test data
y_pred = model2.predict([x_val[:,0:2], x_val[:,2].reshape(-1,1)])

# Evaluate the performance of the XGBoost model
rmse2 = root_mean_squared_error(y_val, y_pred)
# rmse = np.sqrt(mse)
print("RMSE for Embedding LSTM:", rmse2)


8/8 [==============================] - 0s 830us/step
RMSE for Embedding LSTM: tf.Tensor(
[100.74854  100.94575  100.94341  100.95218  100.9235   100.83285
 100.959496 100.095055 100.66446  100.95352  100.160484 100.32992
 100.95227  100.95268  100.949036 100.925156 100.94866  100.86347
 100.63008  100.07765  100.94493  100.58801  100.62995  100.84988
 100.67184  100.67582  100.06561  100.76686  100.95311  100.6292
 100.95993  100.801544 100.889465  99.451065 100.636406  99.45226
 100.67154  100.77766  100.95357  100.94193  100.641    100.8364
  99.55898   98.666595 100.28509   98.66667   99.560074 100.195656
 100.90162  101.12556  100.920555 101.12179  101.08108  100.8373
 101.12001   99.86312  100.91443  101.11358  101.11121  101.12007
 101.09111  101.1172   101.06099  101.098694 101.12437  100.25427
 100.82952  100.3204   100.49158  101.120155 101.11652  100.853035
 101.03048  101.09983  100.79479  100.23669  101.11275  101.12331
 100.752304 100.794655 101.01677  101.039375 100.83696

## XGBoost Model Training

In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [ ]:
#method 2 -- using xgboost for training the model [Only label encoded values]

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Define the XGBoost model
xgb_model1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000,
                             colsample_bytree=0.9, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=3,
                             reg_alpha=1,reg_lambda=1,subsample=0.8
                            )

# Train the XGBoost model
xgb_model1.fit(x_train, y_train)

# Make predictions on the test data
y_pred = xgb_model1.predict(x_val)

# Evaluate the performance of the XGBoost model
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print("RMSE for normal encoded data:", rmse)

RMSE for normal encoded data: 39.506752


In [ ]:
# using xgboost for training the model [One Hot Encoding label encoded values]

xgb_model2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000,
#                              colsample_bytree=0.9, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=3,
#                              reg_alpha=1,reg_lambda=1,subsample=0.8
                            )

# Train the XGBoost model
xgb_model2.fit(x_train_one_hot, y_train_one_hot)

# Make predictions on the test data
y_pred = xgb_model2.predict(x_val_one_hot)

# Evaluate the performance of the XGBoost model
mse = mean_squared_error(y_val_one_hot, y_pred)
rmse2 = np.sqrt(mse)
print("RMSE for one hot encoded data:", rmse2)

RMSE for one hot encoded data: 22.13102


## Testing the model on Submit.csv

In [ ]:
## Running the model on Unseen Data

## Preparing the TEST data FOR INFERENCE
# Exctracting month and year from the column
submit['Month'] = submit['Month'].apply(lambda x: x.split(" - ")[0])
submit['Year'] = 2019
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sept': 9, 'oct': 10, 'nov': 11, 'dec': 12
}


# use the apply() method to convert the month strings to numeric month values
submit['Month'] = submit['Month'].apply(lambda x: month_map[x.lower()])


## Label Encoding the columns in the unseen data

subject_dict = dict(zip(train_data['Subjects'], train_data['subject_encoded']))
submit['subject_encoded'] = submit['Subject'].map(subject_dict)

# Dropping New subjects which were not present in the training data 
submit['subject_encoded'] = submit['subject_encoded'].fillna(-1)
submit.drop(submit.loc[submit['subject_encoded']==-1].index, inplace=True)

submit

,Month,Subject,Count,Year,subject_encoded
0,5,Astronomy and Astrophysics,0,2019,0.0
1,5,Astrophysics,0,2019,1.0
2,5,Astrophysics - Astrophysics of Galaxies,0,2019,2.0
3,5,Astrophysics - Cosmology and Nongalactic Astro...,0,2019,3.0
4,5,Astrophysics - Earth and Planetary Astrophysics,0,2019,4.0
...,...,...,...,...,...
673,10,Statistics - Applications,0,2019,109.0
674,10,Statistics - Computation,0,2019,110.0
675,10,Statistics - Machine Learning,0,2019,111.0
676,10,Statistics - Methodology,0,2019,112.0


In [ ]:
## label encoding unseen data for prediction

def data_preprocess_wo_one_hot_encoding_unseen(df):

    features = df[['Year', 'Month','subject_encoded']].values

    # Extract the target variable

    target = df['Count'].values
    features = np.asarray(features).astype(np.float32)
    target = np.asarray(target).astype(np.float32)
    return features,target

x_unseen,y_unseen = data_preprocess_wo_one_hot_encoding_unseen(submit)

## Predicting with both the trained model [ LSTM and XGB]

In [ ]:
y_predicted_xgb_le = xgb_model1.predict(x_unseen)
y_predicted_lstm_le = model2.predict([x_unseen[:,0:2],x_unseen[:,2].reshape(-1,1)])

submit['Count_predicted_xgb_le'] = y_predicted_xgb_le.tolist()
submit['Count_predicted_lstm_le'] = y_predicted_lstm_le.tolist()
submit

21/21 [==============================] - 0s 606us/step


,Month,Subject,Count,Year,subject_encoded,Count_predicted_xgb_le,Count_predicted_lstm_le
0,5,Astronomy and Astrophysics,0,2019,0.0,342.611816,[0.4158821702003479]
1,5,Astrophysics,0,2019,1.0,342.611816,[2.244643211364746]
2,5,Astrophysics - Astrophysics of Galaxies,0,2019,2.0,360.307068,[4.022736072540283]
3,5,Astrophysics - Cosmology and Nongalactic Astro...,0,2019,3.0,257.290527,[6.679047107696533]
4,5,Astrophysics - Earth and Planetary Astrophysics,0,2019,4.0,209.755844,[1.9871882200241089]
...,...,...,...,...,...,...,...
673,10,Statistics - Applications,0,2019,109.0,7.942367,[0.42208677530288696]
674,10,Statistics - Computation,0,2019,110.0,7.910758,[-0.04214645177125931]
675,10,Statistics - Machine Learning,0,2019,111.0,7.910758,[-0.011590726673603058]
676,10,Statistics - Methodology,0,2019,112.0,7.910758,[-0.03637347370386124]


In [ ]:
final = submit[['Month','Year','Subject','Count_predicted_lstm_le']]

final.rename(columns={'Count_predicted_lstm_le': 'Count'},
          inplace=True, errors='raise')
# final['Count'] = final['Count'].astype(int)
float_list_to_int = lambda x: int(x[0])
# apply the lambda function to the 'float_list_col' column using apply()
final['Count'] = final['Count'].apply(float_list_to_int)
final

/tmp/ipykernel_3432/212876410.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.rename(columns={'Count_predicted_lstm_le': 'Count'},
/tmp/ipykernel_3432/212876410.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Count'] = final['Count'].apply(float_list_to_int)


,Month,Year,Subject,Count
0,5,2019,Astronomy and Astrophysics,0
1,5,2019,Astrophysics,2
2,5,2019,Astrophysics - Astrophysics of Galaxies,4
3,5,2019,Astrophysics - Cosmology and Nongalactic Astro...,6
4,5,2019,Astrophysics - Earth and Planetary Astrophysics,1
...,...,...,...,...
673,10,2019,Statistics - Applications,0
674,10,2019,Statistics - Computation,0
675,10,2019,Statistics - Machine Learning,0
676,10,2019,Statistics - Methodology,0


In [ ]:
final.to_csv('./final-submit.csv',index=False)